In [ ]:
import zipfile

In [ ]:
import os

for dirpath, dirnames, filenames in os.walk("/content/drive/MyDrive/fridge_oven"):

  print(f"There are {len(dirnames)} directories and {len(filenames)} images in this {dirpath}.")

There are 2 directories and 0 images in this /content/drive/MyDrive/fridge_oven.
There are 2 directories and 0 images in this /content/drive/MyDrive/fridge_oven/test.
There are 0 directories and 10 images in this /content/drive/MyDrive/fridge_oven/test/fridge.
There are 0 directories and 10 images in this /content/drive/MyDrive/fridge_oven/test/oven.
There are 2 directories and 0 images in this /content/drive/MyDrive/fridge_oven/train.
There are 1 directories and 20 images in this /content/drive/MyDrive/fridge_oven/train/fridge.
There are 0 directories and 0 images in this /content/drive/MyDrive/fridge_oven/train/fridge/.ipynb_checkpoints.
There are 1 directories and 25 images in this /content/drive/MyDrive/fridge_oven/train/oven.
There are 0 directories and 0 images in this /content/drive/MyDrive/fridge_oven/train/oven/.ipynb_checkpoints.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_shape = (224, 224)
batch_size = 32

train_dir = "/content/drive/MyDrive/fridge_oven/train"
test_dir = "/content/drive/MyDrive/fridge_oven/test"

train_gen = ImageDataGenerator(rescale = 1/255.)
test_gen = ImageDataGenerator(rescale = 1/255.)


train_data = train_gen.flow_from_directory(train_dir, 
                                     target_size = image_shape, 
                                     batch_size = batch_size, 
                                     class_mode = "binary", 
                                     seed = 42)

test_data = test_gen.flow_from_directory(test_dir, 
                                     target_size = image_shape, 
                                     batch_size = batch_size, 
                                     class_mode = "binary", 
                                     seed = 42)

Found 44 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
mobilenet = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [ ]:
def create_model(model_url, num_classes=2):
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=image_shape+(3,)) # define the input image shape
  
  # Create our own model
  model = tf.keras.Sequential([
    feature_extractor_layer, # use the feature extraction layer as the base
    layers.Dense(1, activation='sigmoid', name='output_layer') # create our own output layer      
  ])

  return model

In [ ]:
mobilenet_model = create_model(mobilenet, num_classes = train_data.num_classes)

In [ ]:
# compile

mobilenet_model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits= True), 
                        optimizer = tf.keras.optimizers.Adam(), 
                        metrics = ["accuracy"])

In [ ]:
train_data.image_shape, test_data.image_shape

((224, 224, 3), (224, 224, 3))

In [ ]:
history = mobilenet_model.fit(train_data, 
                    epochs = 5, 
                    steps_per_epoch = len(train_data), 
                    validation_data = test_data, 
                    validation_steps = len(test_data)
                    )

Epoch 1/5
2/2 [==============================] - 29s 16s/step - loss: 0.9172 - accuracy: 0.2955 - val_loss: 0.7070 - val_accuracy: 0.5500
Epoch 2/5
2/2 [==============================] - 0s 244ms/step - loss: 0.7074 - accuracy: 0.4318 - val_loss: 0.5864 - val_accuracy: 0.7500
Epoch 3/5
2/2 [==============================] - 0s 161ms/step - loss: 0.6055 - accuracy: 0.7045 - val_loss: 0.4874 - val_accuracy: 0.8500
Epoch 4/5
2/2 [==============================] - 0s 155ms/step - loss: 0.4885 - accuracy: 0.7727 - val_loss: 0.4081 - val_accuracy: 0.9000
Epoch 5/5
2/2 [==============================] - 0s 148ms/step - loss: 0.3951 - accuracy: 0.9318 - val_loss: 0.3487 - val_accuracy: 0.9000
